In [1]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

'/home/jupyter/.keras/datasets/auto-mpg.data'

In [3]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)
dataset.isna().sum()

"""To keep this initial tutorial simple drop those rows."""

dataset = dataset.dropna()

"""The `"Origin"` column is really categorical, not numeric. So convert that to a one-hot:"""

dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

"""### Split the data into train and test

Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.
"""



train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)



In [4]:
normed_train_data.columns

Index(['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration',
       'Model Year', 'Europe', 'Japan', 'USA'],
      dtype='object')

In [22]:
def build_model(hp):
  model = keras.Sequential()
  model.add(
      keras.layers.Dense(32, activation='relu', input_shape=[len(train_dataset.keys())]))
  model.add(keras.layers.Dense(
      #hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      units=hp.Int("units", min_value=64, max_value=512, step=16),
      activation=hp.Choice("activation", ["relu", "elu"])))
  model.add(keras.layers.Dense(
      #hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      units=hp.Int("units", min_value=128, max_value=1024, step=16),
      activation=hp.Choice("activation", ["relu", "elu"])))
  model.add(keras.layers.Dense(
      #hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      units=hp.Int("units", min_value=128, max_value=1024, step=16),
      activation=hp.Choice("activation", ["relu", "elu"])))
  model.add(keras.layers.Dense(1, activation='relu'))
  model.compile(loss='mse',optimizer=hp.Choice("optimizer", ["adam", "adadelta"]),)
  return model

In [23]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    overwrite=True,
    max_trials=30,
    #max_epochs=10,
    executions_per_trial=2,
    project_name="sample_search")

In [ ]:
tuner.search(normed_train_data, train_labels, epochs=10, validation_data=(normed_test_data, test_labels))



Trial 7 Complete [00h 00m 04s]
val_loss: 8.599861224492392

Best val_loss So Far: 7.561386481309549
Total elapsed time: 00h 00m 28s

Search: Running Trial #8

Hyperparameter    |Value             |Best Value So Far 
units             |464               |144               
activation        |elu               |elu               
optimizer         |adadelta          |adam              

Train on 314 samples, validate on 78 samples
Epoch 1/10
314/314 [==============================] - 1s 3ms/sample - loss: 599.3728 - val_loss: 637.3239
Epoch 2/10
314/314 [==============================] - 0s 483us/sample - loss: 598.6707 - val_loss: 636.6444
Epoch 3/10
314/314 [==============================] - 0s 341us/sample - loss: 597.9435 - val_loss: 635.9407
Epoch 4/10
314/314 [==============================] - 0s 426us/sample - loss: 597.1971 - val_loss: 635.2135
Epoch 5/10
314/314 [==============================] - 0s 303us/sample - loss: 596.4272 - val_loss: 634.4721
Epoch 6/10
314/314 [=========

In [8]:
best_model = tuner.get_best_models()[0]

In [9]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 368)               12144     
_________________________________________________________________
dense_2 (Dense)              (None, 368)               135792    
_________________________________________________________________
dense_3 (Dense)              (None, 368)               135792    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 369       
Total params: 284,417
Trainable params: 284,417
Non-trainable params: 0
_________________________________________________________________
